In [ ]:
# import packages for Google Colab
# from google.colab import drive
# from google.colab import files
import pandas as pd
import numpy as np
import os
import glob
import itertools
# import tensorflow as tf
import scipy

# to make this notebook's output stable across runs
np.random.seed(100)
# tf.random.set_seed(100)

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt


# For connecting data from Google colab
# drive.mount('/content/drive')

In [ ]:
# Python ≥3.7 is required
import sys
assert sys.version_info >= (3, 7)

# Scikit-Learn ≥0.20 is recommended
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
# import tensorflow as tf
# from tensorflow import keras
# assert tf.__version__ >= "2.0"


In [ ]:
# Load packages for .las and .dlis files
# !pip install lasio
# !pip install dlisio
import lasio
from dlisio import dlis

In [ ]:
# Load Survey logs

# Training wells
well_A_inc = lasio.read("./frac_data/training_data/A_Inclination.las", engine='normal').df()
well_B_inc = lasio.read("./frac_data/training_data/B_Inclination.las", engine='normal').df()
well_C_inc = lasio.read("./frac_data/training_data/C_Inclination.las", engine='normal').df()
well_D_inc = lasio.read("./frac_data/training_data/D_Inclination.las", engine='normal').df()
well_E_inc = lasio.read("./frac_data/training_data/E_Inclination.las", engine='normal').df()
well_F_inc = lasio.read("./frac_data/training_data/F_Inclination.las", engine='normal').df()
well_G_inc = lasio.read("./frac_data/training_data/G_Inclination.las", engine='normal').df()
well_H_inc = lasio.read("./frac_data/training_data/H_Inclination.las", engine='normal').df()

# Test wells
well_T1_inc = lasio.read("./frac_data/test_data/T1_Inclination.las", engine='normal').df()
well_T2_inc = lasio.read("./frac_data/test_data/T2_Inclination.las", engine='normal').df()

well_A_inc.columns

In [ ]:
# Load Basic logs

# Training wells
well_A_log = lasio.read('./frac_data/training_data/A_Logs.las', engine='normal').df()
well_B_log = lasio.read('./frac_data/training_data/B_Logs.las', engine='normal').df()
well_C_log = lasio.read('./frac_data/training_data/C_Logs.las', engine='normal').df()
well_D_log = lasio.read('./frac_data/training_data/D_Logs.las', engine='normal').df()
well_E_log = lasio.read('./frac_data/training_data/E_Logs.las', engine='normal').df()
well_F_log = lasio.read('./frac_data/training_data/F_Logs.las', engine='normal').df()
well_G_log = lasio.read('./frac_data/training_data/G_Logs.las', engine='normal').df()
well_H_log = lasio.read('./frac_data/training_data/H_Logs.las', engine='normal').df()

# Test wells
well_T1_log = lasio.read('./frac_data/test_data/T1_Logs.las', engine='normal').df()
well_T2_log = lasio.read('./frac_data/test_data/T2_Logs.las', engine='normal').df()

well_A_log.columns

In [ ]:
frac = pd.read_csv(r"./frac_data/Fracture_data.csv")
frac

Use "Depth", "ADIPnew", and "AAZInew" for the apparent dip and azimuth for the fracture depth.

In [ ]:
# Identify number of fractures per well
print("number of fractures in Well A is ", len(frac[frac['Well']=='A']))
print("number of fractures in Well B is ", len(frac[frac['Well']=='B']))
print("number of fractures in Well C is ", len(frac[frac['Well']=='C']))
print("number of fractures in Well D is ", len(frac[frac['Well']=='D']))
print("number of fractures in Well E is ", len(frac[frac['Well']=='E']))
print("number of fractures in Well F is ", len(frac[frac['Well']=='F']))
print("number of fractures in Well G is ", len(frac[frac['Well']=='G']))
print("number of fractures in Well H is ", len(frac[frac['Well']=='H']))
print("===============================================")
print("number of fractures in Well T1 is ", "unknown") # test well 1
print("number of fractures in Well T2 is ", "unknown") # test well 2

In [ ]:
# Load 10 dlis files for image data
well_A_dlis, *tail = dlis.load('./frac_data/training_data/Well_A_Image.dlis')
well_B_dlis, *tail = dlis.load('./frac_data/training_data/Well_B_Image.dlis')
well_C_dlis, *tail = dlis.load('./frac_data/training_data/Well_C_Image.dlis')
well_D_dlis, *tail = dlis.load('./frac_data/training_data/Well_D_Image.dlis')
well_E_dlis, *tail = dlis.load('./frac_data/training_data/Well_E_Image.dlis')
well_F_dlis, *tail = dlis.load('./frac_data/training_data/Well_F_Image.dlis')
well_G_dlis, *tail = dlis.load('./frac_data/training_data/Well_G_Image.dlis')
well_H_dlis, *tail = dlis.load('./frac_data/training_data/Well_H_Image.dlis')

well_T1_dlis, *tail = dlis.load('./frac_data/test_data/Well_T1_Image.dlis')
well_T2_dlis, *tail = dlis.load('./frac_data/test_data/Well_T2_Image.dlis')


In [ ]:
for frame in well_A_dlis.frames:
    # Search through the channels for the index and obtain the units
    for channel in frame.channels:
        if channel.name == frame.index:
            depth_units = channel.units
    print(f'Frame Name: \t\t {frame.name}')
    print(f'Index Type: \t\t {frame.index_type}')
    print(f'Depth Interval: \t {frame.index_min} - {frame.index_max} {depth_units}')
    print(f'Depth Spacing: \t\t {frame.spacing} {depth_units}')
    print(f'Direction: \t\t {frame.direction}')
    print(f'Num of Channels: \t {len(frame.channels)}')
    print(f'Channel Names: \t\t {str(frame.channels)}')
    print('\n\n')


In [ ]:
# Normalization (or scaling) is required (0 to 255 scale)
dlis_wells = [well_A_dlis, well_B_dlis, well_C_dlis, well_D_dlis, well_E_dlis, well_F_dlis, well_G_dlis, well_H_dlis, well_T1_dlis, well_T2_dlis]

for well in dlis_wells:
    print("min and max values of the ", str(well), " is ", np.min(well.object('FRAME','IMAGE').curves()['IMVIEW1']), " and ", np.max(well.object('FRAME','IMAGE').curves()['IMVIEW1']))



In [ ]:
# Create a dataframe for image data
well_images = {}
list_of_well_images = []

for well in dlis_wells:
  print("well is ", well)
  print("index is ", well.channels[0])
  print("data is ", well.channels[1])
  print("===============================")
  curves = well.object('FRAME','IMAGE').curves()
  well_name = "well_" + str(well)[12:-1]
  well_images[well_name] = pd.DataFrame(curves[str(well.channels[1])[8:-1]], index = curves[str(well.channels[0])[8:-1]]).mul(255.0/np.max(curves[str(well.channels[1])[8:-1]])) # image array data (scale to 0 and 255 to maximize the contrast)
  # well_images[well_name] = pd.DataFrame(curves[str(well.channels[1])[8:-1]], index = curves[str(well.channels[0])[8:-1]]) # image array data

print("Done!")

In [ ]:
# Plot well data for visual inspection
# cm_default = 'inferno_r' # similar color scheme to Techlog image logs
cm_default = 'YlOrBr_r' # similar color scheme to Techlog image logs

# plt.figure(figsize=(14,10))
# plt.subplot(181) # GR
# plt.plot(well_A_log.GR, well_A_log.index)
# plt.xlabel("GR [gAPI]", weight="bold")
# plt.ylabel("Depth [m]", weight="bold")
# plt.axis([0, 200, 500, 0])
# plt.subplot(182) # Resistivity
# plt.plot(well_A_log.ILD, well_A_log.index)
# plt.xscale("log")
# plt.xlabel("ILD [OHMM]", weight="bold")
# plt.axis([0.2, 20000, 500, 0])
# plt.yticks([])
# plt.subplot(183) # Neutron and Density
# plt.plot(well_A_log.NPOR_LIM, well_A_log.index)
# plt.xlabel("Neutron [v/v]", weight="bold")
# plt.axis([0.45, -0.15, 500, 0])
# plt.yticks([])
# plt.subplot(184) # Neutron and Density
# plt.plot(well_A_log.RHO/1000, well_A_log.index)
# plt.xlabel("Density [g/cm3]", weight="bold")
# plt.axis([1.71, 2.71, 500, 0])
# plt.yticks([])
# plt.subplot(185) # Sonic
# plt.plot(well_A_log.DT/3.28084, well_A_log.index) # unit conversion from us/m to us/ft
# plt.xlabel("DTC [us/ft]", weight="bold")
# plt.axis([200, 0, 500, 0])
# plt.title("Well A", weight="bold", fontsize=15)
# plt.yticks([])
# plt.subplot(186) # Azimuth
# plt.plot(well_A_inc.HAZI, well_A_inc.index)
# plt.axis([0, 360, 500, 0])
# plt.xlabel("Azi", weight="bold")
# plt.yticks([])
# plt.subplot(187) # Deviation
# plt.plot(well_A_inc.DEV, well_A_inc.index)
# plt.axis([0, 1, 500, 0])
# plt.xlabel("Dev", weight="bold")
# plt.yticks([])
# plt.subplot(188) # image logs
# plt.imshow(well_images["well_A"], extent=[0, 240, np.max(well_images["well_A"].index), np.min(well_images["well_A"].index)], cmap=cm_default, aspect='auto')
# plt.axis([0, 240, 500, 0])
# plt.xlabel("Image Logs", weight="bold")
# plt.yticks([])
# plt.show()



In [ ]:
# An example of fracture on well H at 358.6637m
i = 103
well = frac.Well.iloc[i]
md = frac.Depth.iloc[i]
ang = frac.ADIPnew.iloc[i]
azi = frac.AAZInew.iloc[i]
title = str(i) + "_fracture_well_" + well + "_at_" + str(md) + "m.jpg"

# Generate x values from 0 to 240 (representing 0 to 360 degrees)
num_data_points = 240  # Number of data points in x-axis (for one period)
deg_per_point = 1.5  # Each data point represents 1.5 degrees
total_degrees = num_data_points * deg_per_point  # Total degrees = 360
x_values = np.linspace(0, total_degrees, num_data_points)

amplitude = 0.5 * 8.5 * 0.0254 * np.tan(np.radians(ang))  # 8.5 in diameter borehole, convert from inch to meter
top_depth = md - 1.0
bot_depth = md + 1.0
well_name = "well_" + well

idx_top = (np.abs(well_images[well_name].index - top_depth)).argmin()
idx_bot = (np.abs(well_images[well_name].index - bot_depth)).argmin()
image_data = well_images[well_name].iloc[idx_top:idx_bot]
y_values = amplitude * np.cos((2 * np.pi * (x_values - azi	- 0)) / 360) + md + 0

plt.figure(figsize=(18,8), dpi=300)
plt.subplot(121) # Without fracture line
plt.imshow(image_data, extent=[0, 360, bot_depth, top_depth], cmap='YlOrBr_r', aspect='auto')
plt.title(title, weight="bold")
plt.ylabel("Depth [m]", weight="bold")
plt.axis([0, 360, bot_depth, top_depth])
plt.subplot(122) # With fracture line
plt.imshow(image_data, extent=[0, 360, bot_depth, top_depth], cmap=cm_default, aspect='auto')
plt.plot(x_values, y_values, color='m', linewidth=4, linestyle='dashed')
plt.title(title, weight="bold")
plt.axis([0, 360, bot_depth, top_depth])
plt.show()


Please note that these fractures were selected by an experienced petrophysicist. Some fractures display a full sinusoidal curve, while others are partial. The reference depth for these fractures can vary, such as the inflection point, average depth, or other criteria. To ensure consistency, all fractures were extended to a full sinusoidal curve, with the corresponding depth set at the center of the cosine curve.

In [ ]:
well_images["well_A"].describe()

## <a>5. Exploratory Data Analysis</a>

Let's experiment with different colormaps for visualization. Since the original DLIS image data is in grayscale, applying an appropriate color scheme can make it easier for the human eye to analyze.

In [ ]:
# # current path
# !pwd

In [ ]:
# Test different color schemes for image data
plt.figure(figsize=(18, 8))
plt.subplot(121)
plt.imshow(well_images["well_A"].iloc[23500:24000], extent=[0, 360, 162.0, 161.0], cmap='YlOrBr_r', aspect='auto')
plt.title('YlOrBr_r', weight="bold")
plt.ylabel("Depth [m]", weight="bold")
plt.axis([0, 360, 162, 161])
plt.subplot(122)
plt.imshow(well_images["well_A"].iloc[23500:24000], extent=[0, 360, 162.0, 161.0], cmap='afmhot', aspect='auto')
plt.title('afmhot', weight="bold")
plt.ylabel("Depth [m]", weight="bold")
plt.axis([0, 360, 162, 161])
plt.show()

Maximizing the contrast between high and low resistivity points is crucial. Let's attempt to convert the static images into dynamic ones for better visualization.

### 修改

In [ ]:
# This function will take a static image (dataframe) as an input and return a dynamic image (dataframe)
def static2dynamic(image_data):
    q3 = np.nanquantile(image_data[image_data < 255.0], 0.03)
    q97 = np.nanquantile(image_data[image_data < 255.0], 0.97)
    image_data_dynamic = image_data.apply(lambda x: 255 * (x - q3) / (q97 - q3))
    image_data_dynamic[image_data_dynamic < 0] = 0
    image_data_dynamic[image_data_dynamic > 255] = 255
    return image_data_dynamic

In [ ]:
import math

def dynamic_image_processing(image_data, min_window_size=10, window_ratio=0.1, overlap_ratio=0.7):
    image_data = image_data.values  # 将 DataFrame 转换为 numpy 数组
    height, width = image_data.shape
    # 自适应计算window_size和step_size
    window_size = max(min_window_size, int(height * window_ratio))
    if window_size > height:
        window_size = height
    step_size = int(window_size * (1 - overlap_ratio))
    if step_size < 1:
        step_size = 1
    # 初始化动态图像数组
    dynamic_image = np.zeros_like(image_data)
    i = 0
    while i < height:
        window_start = i
        window_end = i + window_size
        if window_end > height:
            window_end = height
        current_window_size = window_end - window_start
        window = image_data[window_start:window_end, :]
        dynamic_window = static2dynamic(pd.DataFrame(window)).values  # 将 DataFrame 转换为 numpy 数组
        # 处理窗口间的重叠部分
        if i > 0:
            overlap_start = i - step_size
            if overlap_start < 0:
                overlap_start = 0
            overlap_end = i
            available_overlap = overlap_end - overlap_start
            overlap_size = min(step_size, current_window_size, available_overlap)
            if overlap_size > 0:
                # 生成高斯权重
                sigma = overlap_size / 6  # 标准差，调整以改变曲线形状
                gaussian_weights = [math.exp(-((j - overlap_size/2)**2)/(2*sigma**2)) for j in range(overlap_size)]
                gaussian_weights = np.array(gaussian_weights)
                gaussian_weights /= gaussian_weights.sum()
                for j in range(overlap_size):
                    weight = gaussian_weights[j]
                    dynamic_image[overlap_start + j, :] = (1 - weight) * dynamic_image[overlap_start + j, :] + weight * dynamic_window[j, :]
        # 将动态化后的窗口赋值给动态图像
        dynamic_image[window_start:window_end, :] = dynamic_window
        # 移动步长
        i += step_size
    return dynamic_image

In [ ]:
# A comparison example of static and dynamic images
image_data = well_images['well_D'].iloc[76115:77115]
image_data_dynamic = dynamic_image_processing(image_data)

plt.figure(figsize=(9,4))
plt.subplot(121) # static
plt.imshow(image_data, extent=[0, 360, 220.23, 218.23], cmap='YlOrBr_r', aspect='auto')
plt.title("static", weight="bold")
plt.subplot(122) # dynamic
plt.imshow(image_data_dynamic, extent=[0, 360, 220.23, 218.23], cmap='YlOrBr_r', aspect='auto')
plt.title("dynamic", weight="bold")
plt.show()

In [ ]:
import torchvision.transforms as transforms
from PIL import Image
import cv2
# 定义增强变换
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
])

# 加载图像
image = image_data_dynamic
image_pil = Image.fromarray(image)
# 如果图像模式不是 "RGB"，则转换为 "RGB"
if image_pil.mode != "RGB":
    image_pil = image_pil.convert("RGB")
# 应用增强
augmented_image = transform(image_pil)

# 将 Tensor 转换回 PIL 图像
to_pil = transforms.ToPILImage()
augmented_image_pil = to_pil(augmented_image)

# 显示或保存增强后的图像
augmented_image_pil.show()  # 显示图像
augmented_image_pil.save('augmented_image.jpg')  # 保存图像

Let's create .jpg files with and without fracture lines

In [ ]:
# Create jpg files from fracture images

for i in range(len(frac)):

    well = frac.Well.iloc[i]
    md = frac.Depth.iloc[i]
    ang = frac.ADIPnew.iloc[i]
    azi = frac.AAZInew.iloc[i]
    title = str(i) + "_fracture_well_" + well + "_at_" + str(md) + "m.jpg"

    amplitude = 0.5 * 8.5 * 0.0254 * np.tan(np.radians(ang))  # 8.5 in diameter borehole, convert from inch to meter
    top_depth = md - 1.0
    bot_depth = md + 1.0
    well_name = "well_" + well
    print(title)

    idx_top = (np.abs(well_images[well_name].index - top_depth)).argmin()
    idx_bot = (np.abs(well_images[well_name].index - bot_depth)).argmin()
    image_data = well_images[well_name].iloc[idx_top:idx_bot]

    # Convert static to dynamic image in order to maximize contrast between black and white
    image_data_dynamic = dynamic_image_processing(image_data)
    y_values = amplitude * np.cos((2 * np.pi * (x_values - azi	- 0)) / 360) + md + 0

    plt.figure(figsize=(5,4))
    # plt.figure(figsize=(5,4))
#     plt.subplot(121) # Without fracture line
    plt.imshow(image_data_dynamic, extent=[0, 360, bot_depth, top_depth], cmap='YlOrBr_r', aspect='auto')
    plt.title(title, weight="bold")
#     plt.ylabel("Depth [m]", weight="bold")
    plt.axis([0, 360, bot_depth, top_depth])
#     plt.subplot(122) # With fracture line
#     plt.imshow(image_data_dynamic, extent=[0, 360, bot_depth, top_depth], cmap=cm_default, aspect='auto')
#     plt.plot(x_values, y_values, color='m', linewidth=4, linestyle='dashed')
#     plt.title(title, weight="bold")
#     plt.axis([0, 360, bot_depth, top_depth])
    plt.savefig('./data/images/train/' + title, dpi=300)
    plt.show()
    plt.close()


## <a>6. Build Machine Learning Models</a>






In [ ]:
DATA_DIR = './data/'

In [ ]:
import os
from ultralytics import YOLO
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Be noted that there are a couple of different models (nano(n), small(s), medium(m), large(l), and extra large(x))

model = YOLO('./yolov8n-seg.pt')  # load a pretrained model (recommended for training)
model.train(data ='./config.yaml', epochs = 200, imgsz = 640,amp=False,)

print("Training completed...")

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
# !rm -rf /content/runs # if an error comes up below, remove "runs" directory first
# !scp -r ./runs './fracture_identification'


### **Validation**


Let's review the training results. The figures below show loss, precision, and recall as functions of iterations. Note that the training only runs for about 200 epoch iterations, and performance can be improved with extended training. Additionally, previous studies have shown that small, medium, or larger models outperform the nano model, though they require more training time and computational resources.

In [ ]:
# Read images
import cv2
# from google.colab.patches import cv2_imshow

train_results = cv2.imread("./runs/segment/train14/results.png")
train_results
# OpenCV读取的图像是BGR格式，需要转换为RGB格式以正确显示颜色
train_results_rgb = cv2.cvtColor(train_results, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 5))
plt.imshow(train_results_rgb)


The figures below compare the validation fracture labels with the corresponding predictions and their confidence levels. Some fractures are accurately predicted, while others have low-confidence predictions and are not correctly identified.

In [ ]:
import cv2
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

# Read images
val_labels = cv2.imread('./runs/segment/train6/val_batch0_labels.jpg')
val_predictions = cv2.imread('./runs/segment/train6/val_batch0_pred.jpg')

plt.figure(figsize=(30, 25))
plt.subplot(211)
plt.imshow(cv2.cvtColor(val_labels, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("val_labels")
plt.subplot(212)
plt.imshow(cv2.cvtColor(val_predictions, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("val_predictions")
plt.show()

In [ ]:
# Check validation results
# model_path = '/content/gdrive/My Drive/fracture_identification/run/segment/train/weights/last.pt' # weights from last iteration
model_path = './runs/segment/train14/weights/best.pt' # best weights during the iteration
image_path = './data/images/val/103_fracture_well_H_at_358.6637m.jpg'

img = cv2.imread(image_path)
H, W, _ = img.shape

model = YOLO(model_path)
results = model(img)

for result in results:
    for j, mask in enumerate(result.masks.data):
        
        mask = mask.cpu()
        
        mask = mask.numpy() * 255

        mask = cv2.resize(mask, (W, H))
        cv2.imwrite('./103_fracture_well_H_at_358.6637m_predict.jpg', mask)



In [ ]:
# A function to draw rectangles in the image
import matplotlib.patches as patches

def draw_rect_box(pred_ndarray, depth):
    """
    pred_ndarray: ndarray with a shape of (1200, 1500, 3)
    depth: depth of image (float)
    return: rectangle shape
    """
    x1 = np.min(np.nonzero(pred_ndarray[:,:,1])[1])
    y1 = np.min(np.nonzero(pred_ndarray[:,:,1])[0])
    x_width = np.max(np.nonzero(pred_ndarray[:,:,1])[1]) - np.min(np.nonzero(pred_ndarray[:,:,1])[1])
    y_height = np.max(np.nonzero(pred_ndarray[:,:,1])[0]) - np.min(np.nonzero(pred_ndarray[:,:,1])[0])

    rect0 = patches.Rectangle((200, 143), 1157, 922, linewidth=2, edgecolor='b', facecolor='none')
    rect1 = patches.Rectangle((x1, y1), x_width, y_height, linewidth=1, edgecolor='r', facecolor='none')
    frac_depth = 2/922*(np.max(np.nonzero(pred_ndarray[:,:,1])[0]) + np.min(np.nonzero(pred_ndarray[:,:,1])[0])) / 2 - 1.3102

    return rect0, rect1, frac_depth


In [ ]:
# Detection example
val_img = cv2.imread('./frac_data/images_with_fracture_lines/103_fracture_well_H_at_358.6637m.jpg')
val_pred = cv2.imread('./103_fracture_well_H_at_358.6637m_predict.jpg')

# fig, axs = plt.subplots(1, 2, figsize=(15, 35))
# axs[0].imshow(cv2.cvtColor(val_img, cv2.COLOR_BGR2RGB))
# axs[0].set_title('Original Image')
# axs[0].axis('off')
# axs[1].imshow(cv2.cvtColor(val_pred, cv2.COLOR_BGR2RGB))
rect0, rect1, frac_depth = draw_rect_box(val_pred, depth=358.6637)
# axs[1].add_patch(rect0)
# axs[1].add_patch(rect1)
# axs[1].axis('off')
# axs[1].set_title('Fracture Detection with Rectangle')

print("Answer depth of fracture is", frac.Depth.iloc[103])
print("Predicted depth of fracture is", 358.6637-frac_depth)


In [ ]:
val_pred.shape

In object detection, evaluation can be made with various metrics: mAP (mean Average Precision), IoU (Intersection over union) which measures the overlap between 2 boundaries, AP (Average Precision) indicating the area under the precision-recall curve, or combinations of above.

In our case, which is fracture identification, we will consider F1 score incorporating both recall and precision.

In [ ]:
# Validate the model on your test dataset
metrics_val = model.val(data="./config.yaml")
mp_val, mr_val, map50_val, map_val = metrics_val.box.mean_results()
F1_val = 2 * (mp_val * mr_val) / (mp_val + mr_val)

print("==========================================================")
print("Validation results are ===================================")
print("Mean precision is ", mp_val)
print("Mean recall is ", mr_val)
print("Mean AP at IoU threshold of 0.5 is ", map50_val)
print("Mean AP at IoU thresholds from 0.5 to 0.95 is ", map_val)
print("Mean F1 score is ", F1_val)
print("==========================================================")

As anticipated, our score is relatively low due to several factors, including: (1) insufficient epoch length, (2) use of default hyperparameters, (3) lack of shuffling in the training and validation datasets, (4) possible insufficiency of the YOLO Nano model's complexity, (5) limited size of the training dataset, (6) potential improvements in input data processing, and (7) various other reasons. For actionable steps to enhance performance, refer to the "8. Discussion" section, where we highlight some quick wins.

In [ ]:
# Calculate number of images for test wells using 2 m interval and 1 m overlap
print("Image log depth interval of T1 well is", well_images["well_T1"].index[-1] - well_images["well_T1"].index[0], "m")
print("Image log depth interval of T2 well is", well_images["well_T2"].index[-1] - well_images["well_T2"].index[0], "m")

T1_well_image_number = int(np.floor(well_images["well_T1"].shape[0]/500))
T2_well_image_number = int(np.floor(well_images["well_T2"].shape[0]/500))

print("Total", T1_well_image_number, "images need to be captured from Well_T1")
print("Total", T2_well_image_number, "images need to be captured from Well_T2")
# ignore last half meter of image

In [ ]:
# Create jpg files from T1 test wells (2 m intervals, 1 m overlap)

for i in range(T1_well_image_number):

    md = well_images["well_T1"].index[0] + i + 1.0 # depth is at the center of the image
    title = str(i) + "_test_well_T1" + "_at_" + str(md) + "m.jpg"

    top_depth = md - 1.0
    bot_depth = md + 1.0
    well_name = "well_T1"
    print(title)

    idx_top = i*500
    idx_bot = idx_top + 1000
    image_data = well_images[well_name].iloc[idx_top:idx_bot]

    # Convert static to dynamic image in order to maximize contrast between black and white
    image_data_dynamic = dynamic_image_processing(image_data)

    plt.figure(figsize=(5,4))
    plt.imshow(image_data_dynamic, extent=[0, 360, bot_depth, top_depth], cmap='YlOrBr_r', aspect='auto')
    plt.title(title, weight="bold")
    plt.ylabel("Depth [m]", weight="bold")
    plt.axis([0, 360, bot_depth, top_depth])
    plt.savefig('./frac_data/test_images/T1/' + title, dpi=300)
    plt.show()
    plt.close()

In [ ]:
# Create jpg files from T2 test wells (2 m intervals, 1 m overlap)

for i in range(T2_well_image_number):

    md = well_images["well_T2"].index[0] + i + 1.0 # depth is at the center of the image
    title = str(i) + "_test_well_T2" + "_at_" + str(md) + "m.jpg"

    top_depth = md - 1.0
    bot_depth = md + 1.0
    well_name = "well_T2"
    print(title)

    idx_top = i*500
    idx_bot = idx_top + 1000
    image_data = well_images[well_name].iloc[idx_top:idx_bot]

    # Convert static to dynamic image in order to maximize contrast between black and white
    image_data_dynamic = dynamic_image_processing(image_data)

    plt.figure(figsize=(5,4))
    plt.imshow(image_data_dynamic, extent=[0, 360, bot_depth, top_depth], cmap='YlOrBr_r', aspect='auto')
    plt.title(title, weight="bold")
    plt.ylabel("Depth [m]", weight="bold")
    plt.axis([0, 360, bot_depth, top_depth])
    plt.savefig('./frac_data/test_images/T2/' + title, dpi=300)
    plt.show()
    plt.close()

## <a>7. Prediction</a>
### **Predict depths of potential fractures using images from two test wells (T1 and T2)**

In [ ]:
# Predict fractures from T1 well images
model_path = './runs/segment/train14/weights/best.pt' # best weights during the iteration
folder_path = './frac_data/test_images/T1'

conf_threshold = 0.3 # Users can adjust this confidence threshold number to identify more or less number of fractures
list_of_frac_depth_T1 = []
list_of_confidence_T1 = []

for filename in os.listdir(folder_path):
    if filename.endswith('.jpg'):
        img_path = os.path.join(folder_path, filename)
        depth = float(filename[-10:-5])
        img = cv2.imread(img_path)
        title = filename[:-4] + "_prediction"

        model = YOLO(model_path)
        results = model.predict(img, save=True, save_txt=True, conf=conf_threshold,
                                project="./frac_data/test_predictions/T1/", name=title)

        print("Predicting ", title)
        for r in results:
            boxes = r.boxes
            for box in boxes:
                frac_depth = depth + 2/922*(float(box.xyxy[0][1]) + float(box.xyxy[0][3])) / 2 - 1.3102
                if float(results[0].boxes.conf[0]) > conf_threshold:
                    list_of_frac_depth_T1.append(frac_depth)
                    list_of_confidence_T1.append(float(results[0].boxes.conf[0]))


In [ ]:
# Predict fractures from T2 well images
model_path = './runs/segment/train14/weights/best.pt' # best weights during the iteration
folder_path = './frac_data/test_images/T2'

conf_threshold = 0.3 # Users can adjust this confidence threshold number to identify more or less number of fractures
list_of_frac_depth_T2 = []
list_of_confidence_T2 = []

for filename in os.listdir(folder_path):
    if filename.endswith('.jpg'):
        img_path = os.path.join(folder_path, filename)
        depth = float(filename[-10:-5])
        img = cv2.imread(img_path)
        title = filename[:-4] + "_prediction"

        model = YOLO(model_path)
        results = model.predict(img, save=True, save_txt=True, conf=conf_threshold,
                                project="./frac_data/test_predictions/T2/", name=title)

        print("Predicting ", title)
        for r in results:
            boxes = r.boxes
            for box in boxes:
                frac_depth = depth + 2/922*(float(box.xyxy[0][1]) + float(box.xyxy[0][3])) / 2 - 1.3102
                if float(results[0].boxes.conf[0]) > conf_threshold:
                    list_of_frac_depth_T2.append(frac_depth)
                    list_of_confidence_T2.append(float(results[0].boxes.conf[0]))


It seems the predicted fracture depth is close enough to the real fracture depth.

In [ ]:
# Let's see one of the predictions from test wells

test_image1 = cv2.imread('./frac_data/test_images/T2/412_test_well_T2_at_548.0m.jpg') # , cv2.IMREAD_GRAYSCALE
test_pred1 = cv2.imread('./frac_data/test_predictions/T2/412_test_well_T2_at_548.0m_prediction2/image0.jpg')
test_image1 = cv2.cvtColor(test_image1, cv2.COLOR_BGR2RGB)
test_pred1 = cv2.cvtColor(test_pred1, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15, 10))
plt.subplot(121)
plt.imshow(test_image1)
plt.title("Test image input")
plt.axis("off")
plt.subplot(122)
plt.imshow(test_pred1)
plt.title("Fracture prediction")
plt.axis("off")
plt.show()


The YOLO model predicts two fractures in the example above. However, the one with a confidence score of 0.52 is questionable due to its excessively large bounding box. The other prediction, with a confidence score of 0.80, seems more accurate but only detects a partial sinusoidal curve. Therefore, participants should carefully review all prediction results and select 9 fractures for the T1 well and 11 fractures for the T2 well.

In [ ]:
# Let's see another prediction example from test wells

test_image1 = cv2.imread('./frac_data/test_images/T1/354_test_well_T1_at_455.4m.jpg') # , cv2.IMREAD_GRAYSCALE
test_pred1 = cv2.imread('./frac_data/test_predictions/T1/354_test_well_T1_at_455.4m_prediction2/image0.jpg')
test_image1 = cv2.cvtColor(test_image1, cv2.COLOR_BGR2RGB)
test_pred1 = cv2.cvtColor(test_pred1, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15, 10))
plt.subplot(121)
plt.imshow(test_image1)
plt.title("Test image input")
plt.axis("off")
plt.subplot(122)
plt.imshow(test_pred1)
plt.title("Fracture prediction")
plt.axis("off")
plt.show()


In this instance, a single fracture was divided into multiple segments, and our YOLO model identified each segment separately.

Let's sort the detected fractures by confidence level and select the top 15 for the T1 well and the top 15 for the T2 well, respectively. These numbers can be determined by each participant.

In [ ]:
# Sort detected fractures using confidence level
list_of_confidence_T1_sort, list_of_frac_depth_T1_sort = zip(*sorted(zip(list_of_confidence_T1, list_of_frac_depth_T1), reverse=True))
list_of_confidence_T2_sort, list_of_frac_depth_T2_sort = zip(*sorted(zip(list_of_confidence_T2, list_of_frac_depth_T2), reverse=True))

# There are unknown numbers of fractures on T1 and T2 wells.
# Therefore, we can choose some of detected fractures with highest confidence among 439 from T1 well
# Similarly, we can choose some of detected fractures with highest confidence among 575 from T2 well
print("Depths of 15 most probable fractures on T1 well are: ", list_of_frac_depth_T1_sort[:15])
print("Depths of 15 most probable fractures on T2 well are: ", list_of_frac_depth_T2_sort[:15])

# The above does NOT imply that there are 15 fractures in both the T1 and T2 wells.
# Keep in mind, there are many other smart ways to select these fractures — feel free to get creative!

In [ ]:
# Test T1 Well Image log and detected fractures with confidence
plt.figure(figsize=(9, 5))
plt.subplot(121)
plt.imshow(well_images["well_T1"], extent=[0, 360, np.max(well_images["well_T1"].index), np.min(well_images["well_T1"].index)], cmap=cm_default, aspect='auto')
plt.title("Well T1", weight="bold")
plt.xlabel("Image log", weight="bold")
plt.ylabel("Depth [m]", weight="bold")
plt.axis([0, 360, 480, 80])
plt.subplot(122)
plt.scatter(list_of_confidence_T1, list_of_frac_depth_T1, label="Detected frac.")
plt.scatter(list_of_confidence_T1_sort[:15], list_of_frac_depth_T1_sort[:15], color='red', label="Top 15 frac.") # 15 fractures with highest confidence
plt.title("Well T1", weight="bold")
plt.xlabel("Frac confidence", weight="bold")
plt.axis([0.5, 1, 480, 80])
plt.yticks([])
plt.legend(loc='upper right')
plt.show()


In [ ]:
# Test T2 Well Image log and detected fractures with confidence
plt.figure(figsize=(9, 5))
plt.subplot(121)
plt.imshow(well_images["well_T2"], extent=[0, 360, np.max(well_images["well_T2"].index), np.min(well_images["well_T2"].index)], cmap=cm_default, aspect='auto')
plt.title("Well T2", weight="bold")
plt.xlabel("Image log", weight="bold")
plt.ylabel("Depth [m]", weight="bold")
plt.axis([0, 360, 600, 120])
plt.subplot(122)
plt.scatter(list_of_confidence_T2, list_of_frac_depth_T2, label="Detected frac.")
plt.scatter(list_of_confidence_T2_sort[:15], list_of_frac_depth_T2_sort[:15], color='red', label="Top 15 frac.") # 15 fractures with highest confidence
plt.title("Well T2", weight="bold")
plt.xlabel("Frac confidence", weight="bold")
plt.axis([0.5, 1, 600, 120])
plt.yticks([])
plt.legend(loc='upper right')
plt.show()


In [ ]:
# Intentionally blank here

In [ ]:
# Make a dataframe for the final output
number_of_T1_frac = 15
number_of_T2_frac = 15
Index = list(range(number_of_T1_frac + number_of_T2_frac))
Well = ['T1'] * number_of_T1_frac + ['T2'] * number_of_T2_frac
Depth = list(list_of_frac_depth_T1_sort[:number_of_T1_frac]) + list(list_of_frac_depth_T2_sort[:number_of_T2_frac])

df_output = {'Index': Index, 'Well': Well, 'Depth': Depth}
df_output = pd.DataFrame(df_output)
df_output

Generate the CSV file named "submission.csv". Your CSV file should contain any number of rows between 1 and 99: the first n rows should list the depths of fractures for the T1 well, and remaining rows should list those for the T2 well. Please follow the same format as provided above. Then compress the csv file as a zip file with any name, preferably your own `team_name`.

Upload your zip file (`{team_name}.zip`) to the competition page on Codalab (https://codalab.org/). Details will be announced via email. Your score and ranking will be calculated and displayed in real-time.

The submission file must be a zip file with whatever name. However, the name of the csv file inside the zip has to be "submission.csv". (the csv file should be in the same format as “example_1.csv” file on the GitHub page in terms of number of columns, as well as exactly the same column names)

The submission status might need 10 seconds to be updated, please don't refresh the page too often. Users can find their scores through clicking a button named "Download output from scoring step". The users need to manually submit their best results to the leaderboard. Click "Participate" -> " Submit / View Results", click the "+" symbol in your submission. Be noted that the max submissions per day is 3 and the max submissions total is 100.



In [ ]:
# Making prediction "your_team_name.csv" file

# Replace team_name
team_name = 'WELL_LOGGING_TOP1'

# Write results to csv file
df_output.to_csv('./submissions/submission.csv', index=False)


In [ ]:
# Example of the output score (scores_detail.txt)
# ==================================
# T1 Precision: 0.273
# T1 Recall: 0.333
# T1 F1 Score: 0.300
# T1 rmse: 65.751
# ==================================
# T2 Precision: 0.000
# T2 Recall: 0.000
# T2 F1 Score: 0.000
# T2 rmse: 118.189
# ==================================
# Overall F1 Score: 0.1304
# Overall rmse: 95.6341
# ==================================

## <a>8. Discussion</a>
Let's now discuss the results:

*   We successfully trained the YOLO model using data from 8 wells (A, B, C, D, E, F, G, and H), enabling it to predict potential fractures from image logs.

*   The image logs from the DLIS files are in grayscale. Maximizing image contrast by converting static images to dynamic ones significantly enhances prediction accuracy.

*   Participants should wisely select potential fractures from the T1 and T2 wells.

*   Several low hanging fruits to improve prediction results are described below:

*   Shuffling the training and validation datasets can improve the model's performance.

*   Considering our sample size is relatively small (~100+), data augmentation such as random cropping, flipping, and color adjustments can help too.

*   Dropping uncertain input fracture images from the training dataset can improve prediction performance.

*   Many fractures predicted by the ML model were either false positives or suspicious. Increasing the confidence threshold or applying a filter can help reduce this issue.

*   The YOLO model used in this example was the nano (n) version, which is the fastest but has the lowest performance. Participants are encouraged to test larger models (small (s), medium (m), large (l), and extra large (x)) as well.

*   If you decide not to use pretrained weight and retrain the model, optimize the hyperparameters thoughtfully. This may include adjusting the learning rate, batch size, or number of epochs to find the best settings for the given dataset.

*   Test well predictions will be evaluated using the F1 score. In cases where F1 scores are identical, RMSE will serve as the secondary evaluation metric.




Also, if you want to add more target data, there are several tools to segment your images.

In [ ]:
# Here is a list of tools that you can use for annotating images:

# MakeSense.AI
# LabelImg
# VGG image annotator
# LabelMe
# Scalable
# RectLabel

## <a>9. Summary</a>
In this tutorial, we demonstrate the machine-learning workflow on a practical petrophysical problem: preparing a image log data set, training and testing a prediction model, and finally, testing the model on the blind data to identify fractures. Libraries and open-source tools, such as scikit-learn provide powerful algorithms that can be applied to problems with a few lines of code, which greatly helps to facilitate the research of data science in the petrophysics area. In addition to the procedures mentioned above, many other methods may be applied to improve the performance and stability of the model, such as retraining YOLO model to obtain optimized weights, reprocessing input data, changing model, fine-tunning hyperparameters, and/
or combining them.

Lastly, you can try a fun social experiment: send your friends a few of your image logs, plus your prediction results, and ask them to guess which one is the real fracture!

If you have any questions, please reach out to "pdda_sig@spwla.org".

## <a>10. Acknowledgement</a>
We would like to thank ConocoPhillips Canada for the permission to utilize the image log dataset in developing the ML models of this study. Both SPWLA and ConocoPhillips do not accept any liabilities or make any warranty about the provided data.

